<a href="https://colab.research.google.com/github/jogarrid/nuscenes/blob/master/LSTMmodelNuscenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
map_data_split = np.load('/content/gdrive/My Drive/map_data_split.npy')


In [0]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.contrib import rnn

#define constants
time_steps=4
#hidden LSTM units
num_units= 64*2

#learning rate for adam
learning_rate=0.001

#size of batch
batch_size=128

In [0]:
class Data():
   def __init__(self, map_data_split, train = True):
      #Given data for a list of scenes in mapa_data_split, saves the data necessaryfor prediction.    
      
      means = []
      stds = []
      data = [] #list of standardized partitions belonging to all scenes. 
      indexes = [] #Save information regarding which scene, partition, and instance a segment of data belongs to 
      
      for scene_ix in range(map_data_split.shape[0]):
        for partition_ix in range(map_data_split[scene_ix].shape[0]):
          for instance_ix in range(map_data_split[scene_ix][partition_ix].shape[0]):
            traj_x = map_data_split[scene_ix, partition_ix][instance_ix, :, 0]
            traj_y = map_data_split[scene_ix, partition_ix][instance_ix, :, 1]

            if(~np.any(traj_x == -999) and ~np.any(traj_y == -999)):
              mean_x = np.mean(traj_x)
              mean_y = np.mean(traj_y)
              std_x = np.std(traj_x)
              std_y = np.std(traj_y)

              if(std_x > 0.3):
                traj_x = (traj_x - mean_x)/std_x
              else:
                traj_x = traj_x - traj_x
      
              if(std_y > 0.3):
                traj_y = (traj_y - mean_y)/std_y
              else:
                traj_y = traj_y - traj_y
              if(train):   
                for i in range(traj_x.shape[0] - time_steps):
                    means.append([mean_x, mean_y])
                    stds.append([std_x, std_y])
                    data.append([traj_x[i: i+ time_steps +1], traj_y[i: i+ time_steps +1]])
                    indexes.append([scene_ix, partition_ix, instance_ix])
              else:
                    means.append([mean_x, mean_y])
                    stds.append([std_x, std_y])
                    data.append([traj_x[0: time_steps +1], traj_y[0: time_steps +1]])
                    indexes.append([scene_ix, partition_ix, instance_ix])
                

                  

      self.data = np.array(data)
      self.data_copy= list(data)
     
      self.indexes = np.array(indexes)
      self.means = np.array(means)
      self.stds = np.array(stds)

In [0]:
#For now, both train and test sets include all the dataset.
#Division in test, train and validation sets will be done soon
trainSet = Data(map_data_split)
testSet = Data(map_data_split, train = False)

In [0]:
class Model():
  def __init__(self, time_steps, units, batch_size, learning_rate):
    super(Model, self).__init__()
    n_output = 1     #this model does regression on one number
    self.num_units = units
    self.time_steps = time_steps
    self.batch_size = batch_size
    self.graph = tf.Graph()
    with self.graph.as_default():
      #Now that the data is prepared, we start the tensorflow implementation
      #weights and biases of appropriate shape to accomplish above task
      self.out_weights=tf.Variable(tf.random_normal([num_units,n_output]))
      self.out_bias=tf.Variable(tf.random_normal([n_output]))

      #defining placeholders
      #input image placeholder
      self.x=tf.placeholder("float",[None,time_steps,2])
      #input label placeholder
      self.y=tf.placeholder("float",[None,n_output])

      #processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
      self.input=tf.unstack(self.x ,time_steps, 1)

      #defining the network
      self.lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
      self.outputs,_=rnn.static_rnn(self.lstm_layer,self.input,dtype="float32")

      #converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
      self.prediction=tf.matmul(self.outputs[-1], self.out_weights)+self.out_bias

      #loss_function
      self.loss=tf.reduce_mean(tf.losses.mean_squared_error(self.y, self.prediction))
      #optimization
      self.opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)

      
  def train(self, fx_divided, y_regression, directory):
    #fx_divided needs to be  an array of shape samples,  2, time_steps
    los = 0
    iterations = 4000
    self.directory = directory
    with self.graph.as_default():
      init=tf.global_variables_initializer()
      self.saver = tf.train.Saver()
      with tf.Session() as sess:
        sess.run(init)
        for iter in range(iterations):
            indexesBatch = np.random.randint(0, fx_divided.shape[0], self.batch_size)
            
            
            batch_x = np.zeros((self.batch_size, self.time_steps, 2))
            for i in range(self.batch_size):
              batch_x[i] = fx_divided[indexesBatch[i]].T
              
            batch_x = batch_x + np.random.normal(scale = 0.1, size = batch_x.shape)
            #batch_x = fx_divided[indexesBatch].reshape(self.batch_size, self.time_steps, 2)
            batch_y = y_regression[indexesBatch].reshape(self.batch_size,1) 
            batch_y = batch_y + np.random.normal(scale = 0.1, size = batch_y.shape)
  
            sess.run(self.opt, feed_dict={self.x: batch_x, self.y: batch_y})
            los += sess.run(self.loss,feed_dict={self.x:batch_x, self.y: batch_y})
            if iter % 10==0:
                print("For iter ",iter)
                print("Loss average",los/10)
                #Let's see the inputt and the prediction
                pred = sess.run(self.prediction, feed_dict={self.x: batch_x})[0][0]
                los = 0

        self.save_path = self.saver.save(sess, '/content/gdrive/My Drive/model/' + self.directory)
        print(self.save_path)
 
          
  def predict(self, inputt):
    #This code is very slow as you have to access two graphs, for x and for y, before predicting each sample, and to access each model you need to read a file from disk
    with self.graph.as_default():   
      with tf.Session() as sess:
        self.saver.restore(sess, self.save_path)
        inputt = inputt.T.reshape(1, self.time_steps, 2)
        pred = sess.run(self.prediction, feed_dict={self.x: inputt})[0][0]
 
    return pred

In [10]:
train_data = trainSet.data

#Create and train models#
modelx = Model(time_steps, num_units, batch_size, learning_rate)

modelx.train(train_data[:,:, 0:time_steps], train_data[:,0,time_steps],'modelx1') #Using samples 0-3, we predict sample 4

modely = Model(time_steps, num_units, batch_size, learning_rate)

modely.train(train_data[:,:, 0:time_steps], train_data[:,0,time_steps],'modely1') #Using samples 0-3, we predict sample 4
  

For iter  0
Loss average 0.19496263265609742
For iter  10
Loss average 0.8483531713485718
For iter  20
Loss average 0.3555829465389252
For iter  30
Loss average 0.2247987762093544
For iter  40
Loss average 0.20934377163648604
For iter  50
Loss average 0.20541122779250146
For iter  60
Loss average 0.18448591530323027
For iter  70
Loss average 0.22838916778564453
For iter  80
Loss average 0.19081025198101997
For iter  90
Loss average 0.17066063433885575
For iter  100
Loss average 0.1724285364151001
For iter  110
Loss average 0.189683997631073
For iter  120
Loss average 0.19209535121917726
For iter  130
Loss average 0.18058835119009017
For iter  140
Loss average 0.15929441004991532
For iter  150
Loss average 0.16851118952035904
For iter  160
Loss average 0.16835901364684105
For iter  170
Loss average 0.17276104763150216
For iter  180
Loss average 0.1579444818198681
For iter  190
Loss average 0.17520787939429283
For iter  200
Loss average 0.14695157185196878
For iter  210
Loss average 0.13

In [124]:
#This code is really inefficient: you need to initialize two graphs to make each prediction

n_predict = 6 #predict 3 seconds (6 samples) based on the previous 4 samples (2 seconds)
L = 50
train_Data = testSet.data
stds = testSet.stds
means = testSet.means

data_predicted = np.zeros((L, train_data.shape[1], train_data.shape[2] + n_predict-1)) #added -1
data_predicted[:, :, 0:5] = np.array(train_data_copy)[0:L,:,0:5]
#Create prediction
inputt = np.zeros((2,4))

for i in range(L):
  inputt[:] = train_data[i, :, 0:time_steps] 
  for j in range(n_predict):
    predx= modelx.predict(inputt)
    predy= modely.predict(inputt)
    print(inputt)
    print(predx)
    data_predicted[i,0,time_steps + j] = predx
    data_predicted[i,1,time_steps + j] = predy
    inputt[:,0:(time_steps-1)] = inputt[:,1:(time_steps)]
    inputt[0,time_steps-1] = predx
    inputt[1,time_steps-1] = predy
 
    
  print(data_predicted[i])
  
  data_predicted[i,0,:] = data_predicted[i,0,:]*(stds[i,0]) + means[i,0]
  data_predicted[i,1,:] = data_predicted[i,1,:]*(stds[i,1]) + means[i,1]

  #add mean 
  #Measure mean squared error 

INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/model/modelx1
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/model/modely1
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]]
-0.0037943125
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/model/modelx1
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/model/modely1
[[ 0.          0.          0.         -0.00379431]
 [ 0.          0.          0.          0.03723454]]
-0.00981617
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/model/modelx1
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/model/modely1
[[ 0.          0.         -0.00379431 -0.00981617]
 [ 0.          0.          0.03723454  0.08758759]]
-0.019231737
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/model/modelx1
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/model/modely1
[[ 0.         -0.00379431 -0.00981617 -0.01923174]
 [ 0.          0.03723454

In [137]:
data_predicted.shape

(100, 2, 11)

In [0]:
    predx= modelx.predict(inputt)

In [0]:
np.save('/content/gdrive/My Drive/indexes', testSet.indexes)

In [0]:
np.save('/content/gdrive/My Drive/data_predicted1', data_predicted)